In [1]:
import pandas as pd
import os

votes = pd.read_csv('../physics_csv/Votes.csv')

print(os.getcwd())

# how many userid = -1, they are deleted users
userid_minus1_count = (votes['UserId'] == -1).sum()
print(f"\n#UserId=-1 : {userid_minus1_count}")

# #votes type =8 or 9
bounty_start = votes[votes['VoteTypeId'] == 8]
bounty_close = votes[votes['VoteTypeId'] == 9]

bounty_start_missing = bounty_start[['UserId', 'BountyAmount']].isna().sum()
print("\n how many of Bounty Start (VoteTypeId=8) are missing, if they have a related user")
print(bounty_start_missing)

bounty_close_missing = bounty_close[['UserId', 'BountyAmount']].isna().sum()
print("\n how many of Bounty Close (VoteTypeId=9) are missing, if they have a related user")
print(bounty_close_missing)

# vote type distribution of bountyamount
has_bounty = votes['BountyAmount'].notna()
bounty_types = votes[has_bounty]['VoteTypeId'].value_counts()
print("\nvote type distribution of bountyamount:")
print(bounty_types)


# VoteTypeId=8 and UserId=-1 
vote_type_8_deleted_user = votes[(votes['VoteTypeId'] == 8) & (votes['UserId'] == -1)].shape[0]
print(f"VoteTypeId=8 (bountystart) and UserId=-1 (user deleted): {vote_type_8_deleted_user}")


/Users/I761627/Downloads/Uni/Data Mining Lab/Data-Mining-Stackexchange/EDA/week3

#UserId=-1 : 360

 how many of Bounty Start (VoteTypeId=8) are missing, if they have a related user
UserId          0
BountyAmount    0
dtype: int64

 how many of Bounty Close (VoteTypeId=9) are missing, if they have a related user
UserId          6061
BountyAmount    2293
dtype: int64

vote type distribution of bountyamount:
VoteTypeId
8    6074
9    3768
Name: count, dtype: int64
VoteTypeId=8 (bountystart) and UserId=-1 (user deleted): 360


In [ ]:
votes['UserId'] = votes['UserId'].astype('Int64')
votes['BountyAmount'] = votes['BountyAmount'].astype('Int64')
votes.to_csv('../physics_csv/Votes_cleaned.csv', index=False)

In [2]:
vote_type_map = {
    1: 'AcceptedByOrginator',
    2: 'Upvote',
    3: 'Downvote',
    4: 'Offensive',
    5: 'Favorite',
    6: 'Close',
    7: 'Reopen',
    8: 'BountyStart',
    9: 'BountyClose',
    10: 'Deletion',
    11: 'Undeletion',
    12: 'Spam',
    15: 'ModeratorReview',
    16: 'ApprovedEditSuggestion'
}
vote_counts = votes.groupby(['PostId', 'VoteTypeId']).size().unstack(fill_value=0).reset_index()

# rename the columns
vote_counts.columns = ['PostId'] + [f'{vote_type_map.get(col, f"VoteType_{col}")}_count' for col in vote_counts.columns[1:]]


print(vote_counts.head())


   PostId  AcceptedByOrginator_count  Upvote_count  Downvote_count  \
0       1                          0            89               2   
1       2                          0            32               1   
2       3                          0            46               0   
3       4                          0            30               2   
4       5                          1            53               1   

   Offensive_count  Favorite_count  Close_count  Reopen_count  \
0                0               0            0             0   
1                0               0            0             0   
2                0               0            0             0   
3                0               0            0             0   
4                0               0            0             0   

   BountyStart_count  BountyClose_count  Deletion_count  Undeletion_count  \
0                  0                  0               0                 0   
1                  0              

In [4]:
posts_with_sentiment = pd.read_csv('../physics_csv/Posts_with_sentiment.csv')
# merge with Posts using PostId
merged_df = posts_with_sentiment.merge(vote_counts, left_on='Id', right_on='PostId', how='left')

# delete duplicated PostId column
merged_df = merged_df.drop(columns=['PostId'])

# fill with 0
merged_df = merged_df.fillna(0).astype(int, errors='ignore')

merged_df.to_csv('../physics_csv/Posts_with_vote_counts.csv', index=False)

print(merged_df.head())


   Id  PostTypeId  AcceptedAnswerId             CreationDate  Score  \
0   1           1                 5  2010-11-02 18:59:01.140     87   
1   2           1              5704  2010-11-02 19:09:48.680     31   
2   3           1               180  2010-11-02 19:11:05.857     46   
3   4           2                 0  2010-11-02 19:12:26.543     28   
4   5           2                 0  2010-11-02 19:15:20.813     52   

   ViewCount                                               Body  OwnerUserId  \
0      20982  I often hear about subatomic particles having ...            7   
1       3946  How would you explain string theory to non-phy...           23   
2       3308  This is a question that has been posted at man...           40   
3          0  Imagine going to the rest frame of a massive p...           20   
4          0  Spin is a technical term specifically referrin...           13   

   LastEditorUserId             LastEditDate  ... Favorite_count Close_count  \
0           

In [21]:
print(posts_with_sentiment.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576958 entries, 0 to 576957
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Id                     576958 non-null  int64  
 1   PostTypeId             576958 non-null  int64  
 2   AcceptedAnswerId       576958 non-null  float64
 3   CreationDate           576958 non-null  object 
 4   Score                  576958 non-null  int64  
 5   ViewCount              576958 non-null  float64
 6   Body                   576958 non-null  object 
 7   OwnerUserId            555999 non-null  float64
 8   LastEditorUserId       282657 non-null  float64
 9   LastEditDate           292935 non-null  object 
 10  LastActivityDate       576958 non-null  object 
 11  Title                  234152 non-null  object 
 12  Tags                   234152 non-null  object 
 13  AnswerCount            576958 non-null  float64
 14  CommentCount           576958 non-nu

In [22]:
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576958 entries, 0 to 576957
Data columns (total 39 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Id                            576958 non-null  int64  
 1   PostTypeId                    576958 non-null  int64  
 2   AcceptedAnswerId              576958 non-null  float64
 3   CreationDate                  576958 non-null  object 
 4   Score                         576958 non-null  int64  
 5   ViewCount                     576958 non-null  float64
 6   Body                          576958 non-null  object 
 7   OwnerUserId                   555999 non-null  float64
 8   LastEditorUserId              282657 non-null  float64
 9   LastEditDate                  292935 non-null  object 
 10  LastActivityDate              576958 non-null  object 
 11  Title                         234152 non-null  object 
 12  Tags                          234152 non-nul